In [3]:
import torch
import torch.nn as nn

In [4]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [5]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [6]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [8]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [16]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [10]:
input = torch.randn( 3, 4, 768 )

In [12]:
input

tensor([[[ 1.5852,  0.7124, -2.5290,  ..., -0.2645, -1.0078, -1.9439],
         [ 0.4323, -0.9077,  0.3917,  ..., -0.5338,  0.5965, -0.0693],
         [ 0.6425, -0.7272,  0.3455,  ...,  0.2187, -0.6524,  0.5125],
         [ 1.3831, -0.5528, -1.5321,  ..., -0.2248, -2.2606, -1.5190]],

        [[ 1.0384,  1.2424,  0.3253,  ..., -1.6248, -0.1245,  1.5261],
         [ 0.4452,  1.2925,  1.7482,  ..., -1.4313,  0.0444,  0.7872],
         [ 0.5506, -0.0762, -0.2268,  ...,  0.0315,  0.8042,  0.3774],
         [ 0.9559, -1.6619, -0.0335,  ...,  0.1404,  1.1857, -0.3181]],

        [[-1.2879, -0.8503, -0.8217,  ..., -0.6292, -0.8111, -1.1771],
         [-0.1639, -0.2283, -2.0225,  ...,  0.6507, -0.4333,  0.5057],
         [ 0.5500, -1.0073,  0.9555,  ...,  0.4867,  0.8316,  1.3187],
         [ 0.3468, -0.9812,  0.9767,  ..., -1.6317, -0.4394,  0.2164]]])

In [17]:
output = block( input )

In [18]:
output.shape

torch.Size([3, 4, 768])

In [19]:
output

tensor([[[ 1.8379e+00,  1.1030e+00, -2.4613e+00,  ...,  3.2189e-01,
          -9.5638e-01, -2.2712e+00],
         [ 9.3938e-02, -1.0500e+00,  2.3328e-01,  ..., -5.7315e-02,
           6.0815e-01,  2.4787e-02],
         [ 8.3045e-01, -2.4316e-01,  1.4642e-03,  ...,  5.0329e-01,
          -1.2575e+00,  3.4773e-01],
         [ 1.3165e+00, -2.1253e-01, -1.5922e+00,  ..., -2.2829e-01,
          -2.4800e+00, -1.4317e+00]],

        [[ 1.6071e+00,  9.2813e-01,  2.5890e-01,  ..., -1.6606e+00,
          -3.1930e-01,  1.9111e+00],
         [ 1.2527e+00,  1.4183e+00,  1.5575e+00,  ..., -1.6615e+00,
          -3.5827e-01,  1.0733e+00],
         [ 3.3395e-01, -1.7665e-02,  8.4564e-02,  ..., -2.1349e-02,
           4.6464e-01,  5.8978e-01],
         [ 1.0218e+00, -1.5232e+00,  1.2663e-01,  ..., -1.5243e-01,
           1.0848e+00, -2.6935e-01]],

        [[-1.0797e+00, -7.6656e-01, -7.5902e-01,  ...,  2.1022e-01,
          -3.1099e-01, -1.8383e+00],
         [-1.7802e-01,  3.3552e-01, -2.4010e+00,  .